<a href="https://colab.research.google.com/github/fridaruh/resources_sec_genome/blob/master/LibriSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-087vydhz
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-087vydhz
     |████████████████████████████████| 5.5 MB 14.7 MB/s 
     |████████████████████████████████| 163 kB 66.5 MB/s 
     |████████████████████████████████| 7.6 MB 51.5 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175217 sha256=553b4b5e7f51ebb91b316c5899fa6f7c15fa412b5f8d9c061266f3589ea25f15
  Stored in directory: /tmp/pip-ephem-wheel-cache-_58eptsg/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 15.0 MB/s 
     |████████████████████████████████| 2.2 MB 53.2 MB/s 


# Loading the LibriSpeech dataset

The following will load the test-clean split of the LibriSpeech corpus using torchaudio.

In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class LibriSpeech(torch.utils.data.Dataset):
    """
    A simple class to wrap LibriSpeech and trim/pad the audio to 30 seconds.
    It will drop the last few seconds of a very small portion of the utterances.
    """
    def __init__(self, split="test-clean", device=DEVICE):
        self.dataset = torchaudio.datasets.LIBRISPEECH(
            root=os.path.expanduser("~/.cache"),
            url=split,
            download=True,
        )
        self.device = device

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        audio, sample_rate, text, _, _, _ = self.dataset[item]
        assert sample_rate == 16000
        audio = whisper.pad_or_trim(audio.flatten()).to(self.device)
        mel = whisper.log_mel_spectrogram(audio)
        
        return (mel, text)

In [ ]:
dataset = LibriSpeech("test-clean")
loader = torch.utils.data.DataLoader(dataset, batch_size=16)

  0%|          | 0.00/331M [00:00<?, ?B/s]

# Running inference on the dataset using a base Whisper model

The following will take a few minutes to transcribe all utterances in the dataset.

In [ ]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 46.0MiB/s]


In [ ]:
audio = whisper.load_audio("/content/Entrevista_Thelma.mp3")

In [ ]:
result = model.transcribe(audio)

In [ ]:
transcripcion = result["text"]

Crear el archivo txt

In [ ]:
f= open("Entrevista_Thelma.txt","w+")

In [ ]:
#open text file
text_file = open("/content/Entrevista_Thelma.txt","w")
 
#write string to file
text_file.write(transcripcion)
 
#close file
text_file.close()